In [1]:

from qiskit import QuantumCircuit, transpile
from sympy import Matrix, init_printing
from qiskit.quantum_info import Operator
init_printing()
def op(qc):
    return Matrix(Operator(qc).data.round(3))

n_max = 30

No Ancilla

In [2]:
# for i in range(2, n_max):
#     qc = QuantumCircuit(i)
    
#     qc.mcx(list(range(1, i)), 0)
    
#     qc1 = transpile(qc, basis_gates=["h", "p", "u3", "cx"])
#     print(i, qc1.depth(), qc1.depth(lambda x: x[0].name == "cx"))

1 ancilla (default)

In [3]:
# for i in range(2, n_max):
#     qc = QuantumCircuit(i+1)
    
#     qc.mcx(list(range(1, i)), i, ancilla_qubits=[0], mode="recursion")
    
#     qc1 = transpile(qc, basis_gates=["h", "p", "u3", "cx"])
#     print(i, qc1.depth(), qc1.depth(lambda x: x[0].name == "cx"))

With 1 ancilla (modified)

Based on [Efficient constructions for simulating Multi
Controlled Quantum Gates](https://www.iccs-meeting.org/archive/iccs2022/papers/133530169.pdf) and Qiskit's [MCXRecursive](https://qiskit.org/documentation/_modules/qiskit/circuit/library/standard_gates/x.html#MCXRecursive)

In [4]:
from functools import cache
import numpy as np

def mcx(qc: QuantumCircuit, target, ctrls=None, ancilla=None):
    if ctrls is None or len(ctrls) == 0:
        return qc.x(target)
    if len(ctrls) < 5:
        return qc.mcx(ctrls, target)

    # recurse
    middle = (1 + len(ctrls)) // 2
    first_half, second_half = ctrls[:middle], [*ctrls[middle:], ancilla]

    qc.mcx(first_half, ancilla, second_half[:-1], mode="v-chain-dirty")
    qc.mcx(second_half, target, first_half, mode="v-chain-dirty")
    qc.mcx(first_half, ancilla, second_half[:-1], mode="v-chain-dirty")
    qc.mcx(second_half, target, first_half, mode="v-chain-dirty")
    
@cache
def depth_mcx(i: int) -> tuple[int, int]:
    if i < 1:
        return 0, 0
    
    qc = QuantumCircuit(i+1)
    mcx(qc, 0, list(range(1, i)), i)
    
    # optimization_level=3 doesn't do anything
    qc = transpile(qc, basis_gates=["u", "cx"])
    
    return qc.depth(), sum(qc.count_ops().values()) # qc.depth(lambda x: x[0].name == "cx")
    
for i in range(n_max):
    print(i, *depth_mcx(i))

0 0 0
1 1 1
2 1 1
3 11 14
4 27 30
5 65 77
6 104 119
7 151 171
8 188 223
9 241 275
10 278 327
11 337 379
12 374 431
13 433 483
14 470 535
15 529 587
16 566 639
17 625 691
18 662 743
19 721 795
20 758 847
21 817 899
22 854 951
23 913 1003
24 950 1055
25 1009 1107
26 1046 1159
27 1105 1211
28 1142 1263
29 1201 1315


Depth of Shift operation

In [5]:
@cache
def depth_controlled_shift(i: int):
    if i < 2:
        return ValueError(f"number of qubits must be at least 2, got {i}")
    
    total_depth, gate_count = zip(*(depth_mcx(j) for j in range(2, i+1)))
    return sum(total_depth), sum(gate_count)

for i in range(2, n_max):
    print(i, *depth_controlled_shift(i))

2 1 1
3 12 15
4 39 45
5 104 122
6 208 241
7 359 412
8 547 635
9 788 910
10 1066 1237
11 1403 1616
12 1777 2047
13 2210 2530
14 2680 3065
15 3209 3652
16 3775 4291
17 4400 4982
18 5062 5725
19 5783 6520
20 6541 7367
21 7358 8266
22 8212 9217
23 9125 10220
24 10075 11275
25 11084 12382
26 12130 13541
27 13235 14752
28 14377 16015
29 15578 17330


In [18]:
import numpy as np
from qiskit import QuantumCircuit, transpile

rng = np.random.default_rng(seed=42069)

def c2q_real_depth(n, f=0):
    qc = QuantumCircuit(n+f)
    
    targets = qc.qubits[:n]
    for i, t in enumerate(targets[::-1]):
        j = len(targets) - i
        
        ctrls = qc.qubits[j:]
        
        params = 2*np.pi*rng.random(2**len(ctrls))
        qc.ucry(list(params), ctrls, t)
        
    qc = transpile(qc, basis_gates=["ry", "cx"], optimization_level=3)
    
    # return qc.draw('mpl', reverse_bits=True)
    return qc.depth(), sum(qc.count_ops().values()) # qc.depth(lambda x: x[0].name == "cx")
    
c2q_real_depth(5, 0)

Depth of Convolution operation

In [14]:
def depth_convolution(n: list[int], n_f: list[int], features: int = 0):
    
    ### FIND MAX DIMENSION WITH MAX SIZE FILTER
    n_max, n_f_max = None, None
    for n_i, n_f_i in zip(n, n_f):
        # initial case
        if n_max is None or n_f_max is None:
            n_max, n_f_max = n_i, n_f_i
            
        if n_i > n_max:
            n_max, n_f_max = n_i, n_f_i
        
        if (n_i == n_max) and (n_f_i > n_f_max):
            n_max, n_f_max = n_i, n_f_i
    
    total_depth, gate_count = zip(*(depth_controlled_shift(n_max + 1 - j) for j in range(n_f_max)))
        
    total_depth = sum(total_depth) + 1 # H gate
    gate_count = sum(gate_count) + 3 # SWAP permutations
    
    # U_F depth (FROM TC2022):
    c2q_depth = c2q_real_depth(sum(n_f), features)
    total_depth += c2q_depth[0]
    gate_count += c2q_depth[1]
    # N_f = 2**sum(n_f)
    # total_depth += 3*N_f - sum(n_f) - 4
    # cnot_depth += N_f - 2
    
    return total_depth, gate_count

In [9]:
for i in range(1, n_max):
    print(i, *depth_convolution([i, i, 1], [1, 1, 1], 2))

1 56 60
2 67 74
3 94 104
4 159 181
5 263 300
6 414 471
7 602 694
8 843 969
9 1121 1296
10 1458 1675
11 1832 2106
12 2265 2589
13 2735 3124
14 3264 3711
15 3830 4350
16 4455 5041
17 5117 5784
18 5838 6579
19 6596 7426
20 7413 8325
21 8267 9276
22 9180 10279
23 10130 11334
24 11139 12441
25 12185 13600
26 13290 14811
27 14432 16074
28 15633 17389
29 16871 18756


In [9]:
for j in range(1, 10):
    print(j, *depth_convolution([4, 4], [j, j], 1))

1 116 65
2 201 110
3 403 213
4 1170 598


TypeError: 'ValueError' object is not iterable

Random Junk

In [ ]:
n = 4
qc = QuantumCircuit(n)

qc.mcp(np.pi, list(range(1, n)), 0)
qc.mcry(np.pi, list(range(1, n)), 0)

# qc = transpile(qc, basis_gates=["u", "cx"], optimization_level=3)
display(qc.draw('mpl', reverse_bits=True))

op(qc)

In [ ]:
n = 4
qc = QuantumCircuit(n)

qc.mcp(np.pi, list(range(1, n)), 0)

qc.ry(np.pi/4, 0)
qc.mcx(list(range(1, n-1)), 0)
qc.ry(-np.pi/4, 0)
qc.mcx(list(range(1, n-1)), 0)

qc.cz(n-1, 0)

qc.ry(-np.pi/4, 0)
qc.mcx(list(range(1, n-1)), 0)
qc.ry(np.pi/4, 0)
qc.mcx(list(range(1, n-1)), 0)

qc.cz(n-1, 0)

# qc.p(-np.pi/2, 0)
# qc.mcx(list(range(1, n)), 0)
# qc.p(np.pi/2, 0)
# qc.mcx(list(range(1, n)), 0)

# qc.rz(-np.pi/2, 0)
# qc.mcx(list(range(1, n)), 0)
# qc.rz(np.pi/2, 0)
# qc.mcx(list(range(1, n)), 0)

# qc = transpile(qc, basis_gates=["u", "cx"], optimization_level=1)
display(qc.draw('mpl', reverse_bits=True))

op(qc)